# Batch Scoring

This script does batch scoring.
1. It parses, transforms data in GCS to be scored, 
2. Loads the model in GCS,
3. Uses the model to predict
4. Persists predictions to BigQuery

In [1]:
spark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml import PipelineModel
import sys
from datetime import datetime

In [3]:
# 1a. Arguments
pipelineID = "20220813"
modelVersion = "20220813"
projectNbr = "974925525028"
projectID = "s8s-spark-ml-mlops"
displayPrintStatements = True

In [4]:
# 1b. Variables 
appBaseName = "customer-churn-model"
appNameSuffix = "batch-scoring"
appName = f"{appBaseName}-{appNameSuffix}"
modelBaseNm = appBaseName
bqDatasetNm = f"{projectID}.customer_churn_ds"
modelBucketUri = f"gs://s8s_model_bucket-{projectNbr}/{modelBaseNm}/hyperparameter-tuning/{modelVersion}"
scoreDatasetBucketFQN = f"gs://s8s_data_bucket-{projectNbr}/customer_churn_score_data.csv"
bigQueryOutputTableFQN = f"{bqDatasetNm}.batch_predictions"
scratchBucketUri = f"s8s-spark-bucket-{projectNbr}/{appBaseName}/pipelineId-{pipelineID}/{appNameSuffix}/"
pipelineExecutionDt = datetime.now().strftime("%Y%m%d%H%M%S")

In [5]:
# 1c. Display input and output
if displayPrintStatements:
    print("Starting batch_scoring for Customer Churn Predictions")
    print(".....................................................")
    print(f"The datetime now is - {pipelineExecutionDt}")
    print(" ")
    print("INPUT-")
    print(f"....pipelineID={pipelineID}")
    print(f"....modelVersion={modelVersion}")
    print(f"....projectNbr={projectNbr}")
    print(f"....projectID={projectID}")
    print(f"....displayPrintStatements={displayPrintStatements}")
    print(" ")
    print("OUTPUT-")
    print(f"....BigQuery Table={bigQueryOutputTableFQN}")
    print(f"SELECT * FROM {bigQueryOutputTableFQN} WHERE model_version='{modelVersion}' AND pipeline_id='{pipelineID}' AND pipeline_execution_dt='{pipelineExecutionDt}' LIMIT 10" )
    
    

Starting batch_scoring for Customer Churn Predictions
.....................................................
The datetime now is - 20220802173607
 
INPUT-
....pipelineID=20220813
....modelVersion=20220813
....projectNbr=974925525028
....projectID=s8s-spark-ml-mlops
....displayPrintStatements=True
 
OUTPUT-
....BigQuery Table=s8s-spark-ml-mlops.customer_churn_ds.batch_predictions
SELECT * FROM s8s-spark-ml-mlops.customer_churn_ds.batch_predictions WHERE model_version='20220813' AND pipeline_id='20220813' AND pipeline_execution_dt='20220802173607' LIMIT 10


In [6]:
# 2. Spark Session creation
print('....Initializing spark & spark configs')
spark = SparkSession.builder.appName(appName).getOrCreate()

# Spark configuration setting for writes to BigQuery
spark.conf.set("parentProject", projectID)
spark.conf.set("temporaryGcsBucket", scratchBucketUri)

# Add Python modules
sc.addPyFile(f"gs://s8s_code_bucket-{projectNbr}/pyspark/common_utils.py")
import common_utils

....Initializing spark & spark configs


In [7]:
# 3. Read data to be scored from GCS
print('....Read batch scoring input and profile')
scoreRawDF = spark.read.options(inferSchema = True, header= True).csv(scoreDatasetBucketFQN)
if displayPrintStatements:
    print(scoreRawDF.count())

....Read batch scoring input and profile


411


In [8]:
# 4. Display data, display summary stats
if displayPrintStatements:
    scoreRawDF.show(2)
    scoreRawDF.describe().show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+----------------+--------------+------------+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|   PaymentMethod|MonthlyCharges|TotalCharges|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+----------------+--------------+------------+
|7590-VHVEG|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|            No|         Yes|              No|         No|         No|             No|Month-

+-------+----------+------+-------------------+-------+----------+------------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+
|summary|customerID|gender|      SeniorCitizen|Partner|Dependents|            tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|    MonthlyCharges|      TotalCharges|
+-------+----------+------+-------------------+-------+----------+------------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+
|  count|       411|   411|                411|    411|       411|               411|

In [9]:
# 5. Replace spaces, space with null values in the TotalCharges and MonthlyCharges columns
print('....Data pre-process: fnReplaceSpaceWithNone in TotalCharges and MonthlyCharges')
spaceReplacedDF = common_utils.fnReplaceSpaceWithNone(scoreRawDF)
if displayPrintStatements:
    print(spaceReplacedDF.count())

....Data pre-process: fnReplaceSpaceWithNone in TotalCharges and MonthlyCharges
2022-08-02 17:36:30,169 - common_utils.py - INFO - ....Inside common_utils.fnReplaceSpaceWithNone
411


In [10]:
# 6. Replace non-numeric values in the TotalCharges and MonthlyCharges columns
print('....Data pre-process: ReplaceNotANumberWithNone in TotalCharges and MonthlyCharges')
nanReplacedDF = common_utils.fnReplaceNotANumberWithNone(spaceReplacedDF)
if displayPrintStatements:
    print(nanReplacedDF.count())

....Data pre-process: ReplaceNotANumberWithNone in TotalCharges and MonthlyCharges
2022-08-02 17:36:30,776 - common_utils.py - INFO - ....Inside common_utils.fnReplaceNotANumberWithNone
411


In [11]:
# 7. Drop rows with null in columns
print('....Data pre-process: Drop rows with none')
nullDroppedDF = nanReplacedDF.na.drop()

if displayPrintStatements:
    print(nullDroppedDF.count())

....Data pre-process: Drop rows with none
411


In [12]:
# 8. Replace 'No internet service' across columns to 'No'
print('....Data pre-process: Replace -No internet service- across columns with -No-')
partiallyProcessedDF = common_utils.fnReplaceWithNoForInternetService(nullDroppedDF)
if displayPrintStatements:
    print(partiallyProcessedDF.count())

....Data pre-process: Replace -No internet service- across columns with -No-
2022-08-02 17:36:32,332 - common_utils.py - INFO - ....Inside common_utils.fnReplaceWithNoForInternetService
411


In [13]:
# 9. Add a bin/bucket category for tenure range using Spark SQL and write transformed to dataframe
print('....Data pre-process: Replace -No internet service- across columns with -No-') 
scoreTargetDF = common_utils.fnAddBinForTenure(partiallyProcessedDF, True, spark)
if displayPrintStatements:
    print(scoreTargetDF.count())
    scoreTargetDF.show(2)                            

....Data pre-process: Replace -No internet service- across columns with -No-
2022-08-02 17:36:32,955 - common_utils.py - INFO - ....Inside common_utils.fnAddBinForTenure
411


+----------+------+-------------+-------+----------+------+------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+--------------------+--------------+------------+
|CustomerID|Gender|SeniorCitizen|Partner|Dependents|Tenure|Tenure_Group|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|
+----------+------+-------------+-------+----------+------+------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+--------------------+--------------+------------+
|5027-YOCXN|  Male|            0|    Yes|       Yes|    51|Tenure_48-60|         Yes|          Yes|    Fiber optic|           Yes|         Yes|              No|        Yes|

In [14]:
# 10. Format dataframe names for column name format consistency
scorableDF = scoreTargetDF.select("customerID", "gender", "SeniorCitizen", "Partner", "Dependents", "tenure", "Tenure_Group", "PhoneService", "MultipleLines", "InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "MonthlyCharges", "TotalCharges") \
                                .toDF("customer_id", "gender", "senior_citizen", "partner", "dependents", "tenure", "tenure_group", "phone_service", "multiple_lines", "internet_service", "online_security", "online_backup", "device_protection", "tech_support", "streaming_tv", "streaming_movies", "contract", "paperless_billing", "payment_method", "monthly_charges", "total_charges") 

if displayPrintStatements:
    print(scorableDF.count())
    scorableDF.show(2)

411
+-----------+------+--------------+-------+----------+------+------------+-------------+--------------+----------------+---------------+-------------+-----------------+------------+------------+----------------+--------+-----------------+--------------------+---------------+-------------+
|customer_id|gender|senior_citizen|partner|dependents|tenure|tenure_group|phone_service|multiple_lines|internet_service|online_security|online_backup|device_protection|tech_support|streaming_tv|streaming_movies|contract|paperless_billing|      payment_method|monthly_charges|total_charges|
+-----------+------+--------------+-------+----------+------+------------+-------------+--------------+----------------+---------------+-------------+-----------------+------------+------------+----------------+--------+-----------------+--------------------+---------------+-------------+
| 5027-YOCXN|  Male|             0|    Yes|       Yes|    51|Tenure_48-60|          Yes|           Yes|     Fiber optic|      

In [15]:
# 11. Load the pre-trained, persisted model in GCS
print('....Scoring: Load model out of GCS into memory') 
model = PipelineModel.load(f"{modelBucketUri}/bestModel/")

....Scoring: Load model out of GCS into memory


In [16]:
# 12. Batch scoring
print('....Scoring: Execute model.transform') 
batchScoreResultsDF = model.transform(scorableDF) \
                           .withColumn("model_version", lit(modelVersion)) \
                           .withColumn("pipeline_id", lit(pipelineID)) \
                           .withColumn("pipeline_execution_dt", lit(pipelineExecutionDt)) 

if displayPrintStatements:
    batchScoreResultsDF.show(2)

....Scoring: Execute model.transform


22/08/02 17:37:15 WARN StringIndexerModel: Input column churn does not exist during transformation. Skip StringIndexerModel for this column.
22/08/02 17:37:15 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+------+--------------+-------+----------+------+------------+-------------+--------------+----------------+---------------+-------------+-----------------+------------+------------+----------------+--------+-----------------+--------------------+---------------+-------------+-----------+--------------+-------------------+----------------------+------------+---------------+---------------+------------------+------------------+---------------------+-------------------+----------------------+---------------------+------------------------+--------------------+-----------------------+------------------+---------------------+----------------------+-------------------------+-----------------+--------------------+-----------------+--------------------+---------------------+------------------------+-------------+----------------+----------------------+-------------------------+-------------------+----------------------+--------------------+--------------------+--------------------+

In [17]:
# 13. Persist to BigQuery
print('....Persisting: Batch scoring results to BigQuery')
batchScoreResultsDF.select("customer_id", "gender", "senior_citizen", "partner", "dependents", "tenure", "tenure_group", "phone_service", "multiple_lines", "internet_service", "online_security", "online_backup", "device_protection", "tech_support", "streaming_tv", "streaming_movies", "contract", "paperless_billing", "payment_method", "monthly_charges", "total_charges","prediction","model_version","pipeline_id","pipeline_execution_dt") \
.write.format('bigquery') \
.mode("overwrite")\
.option('table', bigQueryOutputTableFQN) \
.save()


....Persisting: Batch scoring results to BigQuery
